# 版本记录

- v0.0.1  
初始版本

- v0.0.2  
原来的模型不够简洁，去掉匹配规则对象，全部换成标记符对象。  
考虑了一下，还是保留匹配规则对象，因为匹配规则，将来可能是重要的管理单元。  
再考虑了一下，准备通过运算符重载的方式增加匹配规则，暂时先不体现匹配规则对象。  
my parser有两种实现方案：
一种是利用操作符重载简洁的表达，形如：s += a| a&s   
另外一种采用函数方式表达可理解性更好，形如：s.add(a); s.add(a,s)  
这两种方式是等价的，我先用简洁的实现看看方不方便，如果不行再换成函数式  

初步实现了`S=a`的解析。

v0.0.3   
想到一种新的实现方案，结合v0.0.2两个方案的优点。使用操作符重载来简洁表达，相同操作自动合并，兼顾可理解性和可读性。

# 思路

## 描述

- 左侧有且仅有一个非终结符  
    形如S=aS
- 匹配接口IMatch  
    实现了Match函数，Match函数负责识别一段文本，返回可能识别到的多个结果列表，如果没有识别到，返回(False,...)  
- 标记符对象（Symbol）  
    - 实现了IMatch接口  
    - 终结符对象（Terminal）  
    单个标记符对象，不包括其他标记符对象，直接匹配  
    - 复合标记符(MultiSymbol)  
    管理着一个标记符符号列表，包含一个或多个符号
    - 与复合标记符对象(AndSymbol)  
    这是一个复合标记符对象，包含多个标记符，这些标记符之间是与的关系  
    - 或复合标记符对象(OrSymbol)  
    这是一个复合标记符对象，包含多个标记符，这些标记符之间是或的关系  
    - 非终结符标记符对象(NonTerminal)  
      是一种特殊的或复合标记符对象，这些标记符将来作为匹配规则对象来处理  
    - 开始符（Starter）  
        特殊的非终结标记符，匹配就从它开始  
    - 参数化标记符  
        自己管理和处理标记符列表的标记符  

## 对象继承关系

采用graphviz描述，直接运行下面的脚本即可。

In [1]:
def 对象继承关系():
    from graphviz import Digraph
    dot = Digraph()
    # 节点描述
    dot.node('IMatch',xlabel='定义了Match函数')
    dot.node('Symbol',xlabel='标记符')
    dot.node('Terminal',xlabel='终结符')
    dot.node('MultiSymbol',xlabel='复合标记符，管理一个符号列表')
    dot.node('AndSymbol',xlabel='与复合标记符')
    dot.node('OrSymbol',xlabel='或复合标记符')
    dot.node('NonTerminal',xlabel='非终结符,包含一个或多个可称之为规则的标记符')
    dot.node('Starter',xlabel='开始符')
    # 继承关系描述
    dot.edge('IMatch','Symbol')
    dot.edge('Symbol','Terminal')
    dot.edge('Symbol','MultiSymbol')
    dot.edge('MultiSymbol','AndSymbol')
    dot.edge('MultiSymbol','OrSymbol')
    dot.edge('OrSymbol','NonTerminal')
    dot.edge('NonTerminal','Starter')
    # 显示图像
    dot

#对象继承关系()

# 实现

## 引入及类型

In [2]:
from printobject import pp
from types import MethodType 

In [3]:
from typing import List, Tuple

# 匹配之后，剩余的待匹配的结果类型描述，形如(True,[(2,3),(5,6)])，第一项表示是否匹配成功，第2项表示匹配的结果，可能有多个待匹配项
# 每一个匹配项第一项表示起始索引，第2项表示长度
ParseResults = Tuple[bool,List[Tuple[int,int]]]

## 匹配接口IMatch

In [4]:
class IMatch:
    # 表示待解析的原始字符串
    Text :str = None

    def Match(self, startIndex: int, length: int, isFullMatch: bool) -> ParseResults:
        '''
        功能描述：对待解析的字符串进行匹配，返回匹配的结果
        startIndex：待匹配字符串的起始索引
        length：待匹配字符串的长度
        isFullMatch：表示是否需要完全匹配，如果完全匹配，则必须字符串匹配完，才返回成功，只匹配部分，则匹配失败
        返回值：ParseResults类型，可能有多个剩余的待匹配项
        '''
        pass
    pass

## 标记符对象Symbol

In [5]:
class Symbol(IMatch): 
    # 重载&函数
    def __and__(self, dst):
        # 把两个标记符对象合并到一起
        def _与合并(s1,s2):
            if type(s2)==AndSymbol:
                s1.symbolList.extend(s2.symbolList)
            else:
                s1.symbolList.append(s2)
        newSymbol = AndSymbol()
        _与合并(newSymbol,self)
        _与合并(newSymbol,dst)
        return newSymbol
    
    # 重载|函数
    def __or__(self, dst):
        # 把两个标记符对象合并到一起
        def _或合并(s1,s2):
            if type(s2)==OrSymbol:
                s1.symbolList.extend(s2.symbolList)
            else:
                s1.symbolList.append(s2)
        newSymbol = OrSymbol()
        _或合并(newSymbol,self)
        _或合并(newSymbol,dst)
        return newSymbol

## 多匹配函数

是为了合并多个符号对象或者多个结果对象。

In [6]:
def MultiMatch(li: list, isFullMatch: bool, symbol: Symbol = None, startIndex: int = None, length: int = None) -> ParseResults:
    if len(li) <= 0: return (False,None)
    lastSuccess = False
    lastResult = []
    for item in li:
        if symbol != None:
            # 表示多结果匹配
            res = symbol.Match(item[0], item[1], isFullMatch)
        else:
            # 表示多符号匹配
            res = item.Match(startIndex, length, isFullMatch)
        # 如果匹配失败，则继续
        if res[0]==False: continue
        # 如果匹配成功
        lastSuccess = True
        # 合并每个匹配结果
        lastResult.extend(res[1])
    # 匹配结果去重
    lastResult = list(set(lastResult))
    # 返回最终结果
    return (lastSuccess,lastResult)

### 终结符对象Terminal

In [7]:
class Terminal(Symbol):pass

## 复合标记符对象MultiSymbol

In [8]:
class MultiSymbol(Symbol):
    def __init__(self):
        super().__init__()
        # 管理符号对象列表，这些符号对象的关系依赖于对象的类型
        self.symbolList: List[Symbol] = []

## 或复合标记符对象

In [9]:
class OrSymbol(MultiSymbol):
    '''复合标记符对象列表中的标记符之间是或的关系，可以采用多匹配函数进行匹配'''
    def Match(self, startIndex: int, length: int, isFullMatch: bool) -> ParseResults:
        return MultiMatch(self.symbolList, isFullMatch, startIndex=startIndex, length=length)

## 与复合标记符对象

In [10]:
class AndSymbol(MultiSymbol):
    '''复合标记符对象列表中的标记符之间是与的关系，可以采用多匹配函数进行匹配'''
    def Match(self, startIndex: int, length: int, isFullMatch: bool) -> ParseResults:
        if len(self.symbolList) <= 1: return (False,None)
        # 上一个处理结果，先进行初始化
        preResult = [(startIndex,length)]
        # 先处理前面的，最后一个单独处理
        for symbol in self.symbolList[:-1]:
            res = MultiMatch(preResult,False,symbol)
            if res[0]==False: return (False,None)
            preResult = res[1]
        # 再处理最后一个
        return MultiMatch(preResult,isFullMatch,self.symbolList[-1])

## 非终结符对象

形如：A = abc，A = aA，其中A就是非终结符。他管理的符号对象列表也称之为规则，彼此之间是或的关系，所以继承或标记符对象。

In [11]:
class NonTerminal(OrSymbol): 
    def __iadd__(self,dst):
        self.symbolList.append(dst)
        return self

### 关于结束符

开始似乎需要结束符，结束符表示最后空的匹配，才能匹配成功。后来思考一下，似乎不需要结束符了，因为对于每一个匹配规则对象，都要求完全匹配，不允许还剩余没有匹配的部分。所以，对于标记符对象、匹配规则对象，都要检查是否匹配完整。  

表示一个基础匹配单元，包括开始符、非终结标记符、终结标记符。  
```
# S是开始符，`aS`表示S的一个匹配规则，S、A都是非终结标记符，a是终结标记符
S=aS
A=aB
```

## 开始符对象

In [12]:
class Starter(NonTerminal):
    def Run(self, txt: str) -> int:
        '''
        txt：要匹配的文本
        返回匹配成功的次数，如果为0，表示没有匹配成功，如果大于1，表示有多个成功的匹配
        '''
        IMatch.Text = txt
        res = self.Match(0,len(IMatch.Text),True)
        return res[0]

# 应用

## 实现`S=a`

## 字母`a`的终结符对象

In [13]:
class A(Terminal):
    def Match(self, startIndex: int, length: int, isFullMatch: bool) -> ParseResults:
        if length <= 0: return (False,None)
        
        # 如果严格匹配，只能匹配单字符
        if isFullMatch==True and length > 1: return (False,None)
        
        if IMatch.Text[startIndex] != 'a': return (False,None)
        return (True,[(startIndex+1,length-1)])

# 试验运行

In [14]:
def Run():
    s=Starter()
    a = A()
    s += a | a&a
    res = s.Run('aa')
    assert(res==True)

In [15]:
Run()